# Livraisons à domicile

Ce fichier contient plusieurs étapes
1. import de librairies essentielles
2. Insertion du fichier csv de clients pour livraison
3. Détermination de la position gps (geocoding)
3. Traitement du fichier et des doublons
4. Mise en place sur une carte

### Pour déterminer l'ordre des livraisons il faut ajouter l'adresse ci-dessous de la livraison

#### Il est important de laisser les guillements simples '' de part et d'autre des noms
-----

In [1]:
#qgrid.nbinstall(overwrite=True)
import pylab
import qgrid
import pandas as pd
import geopandas as gpd
import geopy
import io
import codecs
import base64
from traitlets import traitlets
from geopy.geocoders import Nominatim, GoogleV3, IGNFrance, BANFrance
from geopy.extra.rate_limiter import RateLimiter
import matplotlib.pyplot as plt
import folium
from folium import plugins, IFrame
from folium.features import *

import random
import csv
import simplekml

from pyroutelib3 import Router
import numpy as np

from scipy.spatial import distance_matrix
from scipy.spatial.distance import cdist
from scipy.sparse.csgraph import shortest_path

from tsp_solver.greedy import solve_tsp
from ipywidgets import interact, interactive, fixed, widgets
from IPython.display import display, HTML, FileLink
from IPython.display import HTML

pylab.rcParams['figure.figsize'] = (24.0, 12.0) #configure the figure

def lon_lat_to_xy(lon, lat, R = 6371000.0):
    """
    calculates lon, lat coordinates of a point on a sphere with
    radius R
    """
    lon_r = np.radians(lon)
    lat_r = np.radians(lat)

    x =  R * np.cos(lat_r[0]) * lon_r
    y = R * lat_r
    return x,y

def get_distance(routeLatLons):
    
    data = []
    for iter, point in enumerate(routeLatLons):
        data.append([point[0], point[1]])
    
    columns = ['Latitude', 'Longitude']
    temp = pd.DataFrame(data, columns=columns)
    
    x_temp,y_temp = lon_lat_to_xy(temp['Longitude'], temp['Latitude'], R = 6371000.0)
    x = x_temp - x_temp[0]
    y = y_temp - y_temp[0]
    d_x = np.diff(x)
    d_y = np.diff(y)
    ds = np.sqrt(d_x*d_x + d_y*d_y) 
    return np.sum(ds)

In [2]:
class LoadedButton(widgets.Button):
    """A button that can holds a value as a attribute."""

    def __init__(self, value=None, *args, **kwargs):
        super(LoadedButton, self).__init__(*args, **kwargs)
        # Create the value attribute.
        self.add_traits(value=traitlets.Any(value))

def find_magasin_address(Nom_Magasin, Prenom_Magasin, Adresse_Magasin, CP_Magasin, Ville_Magasin):
    Nom_Magasin = Nom_Magasin.value
    Prenom_Magasin = 'QUIVABIEN'
    Adresse_Magasin = Adresse_Magasin.value
    CP_Magasin = CP_Magasin.value
    Ville_Magasin = Ville_Magasin.value
    
    pt_depart = pd.DataFrame({'N' : 0, 'Type':['L'], 'Nom':[Nom_Magasin], 'Prenom':[Prenom_Magasin], 'Adresse':[Adresse_Magasin], 'CP':[CP_Magasin], 'Ville':[Ville_Magasin], "Qte_totale":['1000000'], 'Tel' : ['06660'], 'Commentaires' :['']})

    #On affiche l'adresse totale
    pt_depart['addresse_tot'] = pt_depart['Adresse'].astype(str) + ',' + \
                pt_depart['CP'].astype(str) + ',' + \
                pt_depart['Ville'].astype(str) + ', France'

    locator = BANFrance()
    geocode = RateLimiter(locator.geocode, min_delay_seconds=0.5)
    pt_depart['location'] = pt_depart['addresse_tot'].apply(geocode)
    #On rajoute un point avec des valeurs nulles
    pt_depart['point'] = pt_depart['location'].apply(lambda loc: tuple(loc.point) if loc else None)

    #On sépare ensuite l'objet point en latitude, longitude et altitude
    pt_depart[['latitude', 'longitude', 'altitude']] = pd.DataFrame(pt_depart['point'].tolist(), index=pt_depart.index)
    pt_depart['addresse_BAN'] = pt_depart.location.astype(str)
    return pt_depart

def find_livraison(fichier):
    df = pd.read_csv(io.BytesIO(fichier), sep=';')
    df.dropna(subset=['N'], inplace=True)
    df['Type'] = df['Type'].str.lower()
    df['N'] = df['N'].astype('int')
    df['Tel'] = df['Tel'].astype('str')
    df['Tel'] = df['Tel'].str.replace(' ','')
    #df['Tel'] = df['Tel'].astype('int')
    df['Tel'] = '0' + df['Tel'].str.replace(' ','')
    df['Qte_totale'] = df['Qte_totale'].astype('int')
    df['Commentaires'] = df['Commentaires'].replace(np.nan, '', regex=True)
    df['Type'] = df['Type'].apply(lambda x: x[0])
    df['A_livrer'] = True

    liv = df.loc[df['Type'] == 'l']
    liv = liv.copy(deep=True)
    try:
        groupes = pd.read_csv("groupes.csv", sep=';')
        liv = liv.merge(groupes, how='left').fillna(0)
    except:
        liv['Groupe'] = 1
    liv['Groupe'] = liv['Groupe'].astype('int')
    return liv

def draw_map_from_liv(find_button, get_csv_button):
    qgrid_widget = get_csv_button.value
    liv_updated = (qgrid_widget.get_changed_df()).copy()
    liv_updated2 = liv_updated[liv_updated['A_livrer'] == True]
    liv = liv_updated2.copy()

    pt_depart = find_button.value
    #On affiche l'adresse totale
    liv['addresse_tot'] = liv['Adresse'].astype(str) + ', ' + \
                    liv['CP'].astype(str) + ', ' + \
                    liv['Ville'].astype(str) + ', France'   

    #En utilisant un $locator de geopy, on trouve l'adresse et la position
    locator = BANFrance()
    geocode = RateLimiter(locator.geocode, min_delay_seconds=0.2)
    liv['location'] = liv['addresse_tot'].apply(geocode)
    #On rajoute un point avec des valeurs nulles
    liv['point'] = liv['location'].apply(lambda loc: tuple(loc.point) if loc else None)

    #On sépare ensuite l'objet point en latitude, longitude et altitude
    liv[['latitude', 'longitude', 'altitude']] = pd.DataFrame(liv['point'].tolist(), index=liv.index)
    
    #On ajoute l'adresse exacte dans un champ 
    liv['index'] = np.arange(len(liv))
    liv['addresse_BAN'] = liv.location.astype(str)
    liv['addresse_BAN'] = liv['addresse_BAN'].str.split(',', n = 1)
    liv.addresse_BAN = liv.addresse_BAN.str[0]
    #On sépare les commandes
    liv.addresse_BAN[liv.addresse_BAN.duplicated()].head()
    liv['Prenom'].fillna('', inplace=True)

    #dupli = df.duplicated(['addresse_BAN'])
    dupli = liv[liv.duplicated(['Nom', 'addresse_BAN'], keep=False)]

    df2 = liv.groupby(['Nom','addresse_BAN'], as_index=False)[["Qte_totale"]].sum()
    df3 = liv.drop_duplicates(subset =['Nom','addresse_BAN'], keep ="first", inplace = False)
    mergedDf = df2.merge(df3[['index','Nom','Prenom','Adresse','CP','Ville','addresse_BAN','Tel','Commentaires','latitude', 'longitude','Qte_totale','Groupe']], on=['Nom','addresse_BAN','Qte_totale'])

    mergedDf.sort_values(['index'], ascending=1, inplace=True)
    mergedDf.reset_index(drop=True, inplace=True)

    pt_depart['x'], pt_depart['y'] = lon_lat_to_xy(pt_depart['longitude'], pt_depart['latitude'], R = 6371000.0)
    mergedDf['x'], mergedDf['y'] = lon_lat_to_xy(mergedDf['longitude'], mergedDf['latitude'], R = 6371000.0)
    df_array = mergedDf[['x', 'y']].to_numpy()
    dist_map = cdist(df_array, df_array)
    pd.DataFrame(dist_map)[0].values
    to_drop = np.argwhere(pd.DataFrame(dist_map)[0].values > 1.E5)

    #df_lat_nulle = mergedDf[mergedDf['latitude'].isnull()]
    df_lat_nulle = mergedDf[mergedDf['latitude'].isna()]
    for value in to_drop.flatten():
        value_int = int(value)
        df_lat_nulle = df_lat_nulle.append(mergedDf.iloc[value_int]) 
        #mergedDf.drop([value_int])       

    df_lat_nulle.drop_duplicates(subset ='addresse_BAN', keep ="first", inplace = True)

    final_client = pd.concat([mergedDf, df_lat_nulle,df_lat_nulle]).drop_duplicates(keep=False)
    df_lat_nulle.to_csv('clients_erreur.csv', sep=';')
    final_client.to_csv('clients_correct.csv', sep=';')

    tournees=[]
    for region, df_region in final_client.groupby('Groupe'):

        #df_region.to_csv('clients_tournee_' + str(region) + '.csv', sep=';')    
        #path = solve_tsp(dist_map_gps, endpoints = (0,-1))

        df_array = df_region[['x', 'y']].to_numpy()
        #pisciculture debut et fin
        df_array = np.vstack(((pt_depart.iloc[0]['x'], pt_depart.iloc[0]['y']),df_array))
        df_array = np.vstack((df_array, (pt_depart.iloc[0]['x'], pt_depart.iloc[0]['y'])))
        dist_map = cdist(df_array, df_array)
        path = solve_tsp(dist_map, endpoints = (0,-1))  
        path_red = path[1:-1]
        sorterIndex = dict(zip(path_red,range(1,len(path_red)+1)))
        df_region_ordered = df_region.copy(deep=True)    
        df_region_ordered['index'] = np.arange(1,len(path_red)+1)
        df_region_ordered['order'] = df_region_ordered['index'].map(sorterIndex)    
        df_region_ordered.sort_values(['order'], ascending=1, inplace=True)
        df_region_ordered.reset_index(drop=True, inplace=True)            
        df_region_reduced = df_region_ordered[["Nom", "Prenom", "Tel", "Adresse", "CP", "Ville", "Qte_totale"]]
        df_region_reduced.to_csv('clients_tournee_' + str(int(region)) + '.csv', sep=';')
        tournees.append(df_region_ordered)
        
    pt_depart['order'] = 0
    pt_depart['Groupe'] = 0
    final_client = pd.concat([pt_depart, final_client], ignore_index=True, join="inner")

    for index, tournee in zip(range(len(tournees)),tournees):
        tournees[index] = pd.concat([pt_depart, tournee], ignore_index=False, join="inner")        
        
    folium_map = folium.Map(location=[final_client['latitude'].mean(), 
                                        final_client['longitude'].mean()], 
                                        zoom_start=10, tiles='OpenStreetMap')

    folium.plugins.BoatMarker([final_client.iloc[0]['latitude'], final_client.iloc[0]['longitude']]).add_to(folium_map)

    Tournee_group=[]

    color_icons = ['red', 'blue', 'green', 'orange', 'darkblue', 'darkgreen', 'black', 'lightgreen', 'beige', 'pink', 'lightgray', 'lightred', 'purple', 'darkred', 'darkpurple', 'cadetblue', 'gray', 'lightblue']

    for number, tournee in zip(range(len(tournees)),tournees):

        Tournee = FeatureGroup(name='Tournee'+str(int(number)))
        for i in range(1,len(tournee)):

            url='https://www.google.fr/maps/dir//' + \
            str(tournee.iloc[i]['latitude'])+ ',' + str(tournee.iloc[i]['longitude'])
            address = str(tournee.iloc[i]['Adresse']) + ", " + str(tournee.iloc[i]['CP']) + ", " + str(tournee.iloc[i]['Ville'])
            nom_prenom = str(tournee.iloc[i]['Nom']) + '\t' + str(tournee.iloc[i]['Prenom'])
            tel = '0' + str(tournee.iloc[i]['Tel'])
            colis = str(tournee.iloc[i]['Qte_totale']) + ' bieres'
            commentaires = str(tournee.iloc[i]['Commentaires'])
            lien = '<a href=' + url + '> Maps </a>'

            popup_name = '<b>' + nom_prenom + '</b>' + '<br>' + str(address) + '<br>'  + tel + '<br>' + colis + '<br>' + commentaires + '<br>' + lien

            popup = folium.Popup(popup_name, max_width=650)

            icon_order = DivIcon(
                icon_size=(150,36),
                icon_anchor=(0,0),
                html='<div style="font-size: 14pt">' + str(int(tournee.iloc[i]['order'])) + '</div>',
                )

            icon = folium.map.Icon(color=color_icons[number%len(color_icons)])        
            folium.Marker([tournee.iloc[i]['latitude'], tournee.iloc[i]['longitude']], icon=icon_order).add_to(Tournee)
            folium.Marker([tournee.iloc[i]['latitude'], tournee.iloc[i]['longitude']], icon=icon, popup=popup).add_to(Tournee)
            Tournee_group.append(Tournee)

    for tournee in Tournee_group:
            folium_map.add_child(tournee)

        #FastMarkerCluster(data=list(zip(df['latitude'].values, df['longitude'].values)), popup=df['addresse_tot']).add_to(folium_map)

        #folium_map.add_child(feature_group)
    folium.LayerControl().add_to(folium_map)
#    folium_map.save("map_tournees.html")
    return folium_map        

In [3]:
Nom_Magasin = widgets.Text(
    value='MAGASIN',
    placeholder='Nom du Magasin',
    description='Nom du Magasin:',
    disabled=False
)

Prenom_Magasin = 'QUIVABIEN'

Adresse_Magasin = widgets.Text(
    value='71, en Fournirue',
    placeholder='Adresse du Magasin',
    description='Adresse du Magasin:',
    disabled=False
)

CP_Magasin = widgets.Text(
    value='57000',
    placeholder='Code Postal',
    description='Code Postal:',
    disabled=False
)

Ville_Magasin = widgets.Text(
    value='Metz',
    placeholder='Ville',
    description='Ville:',
    disabled=False
)

find_button = LoadedButton(
    value=1,
    description='Trouver Magasin',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Position initiale Magasin',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
output_find = widgets.Output(layout=widgets.Layout(border='1px solid black', overflow_y='auto', height='30px'))

def on_find_button_clicked(b):
    with output_find:
        b.value = find_magasin_address(Nom_Magasin, Prenom_Magasin, Adresse_Magasin, CP_Magasin, Ville_Magasin)
        output_find.clear_output()
        print('Found : ' +  b.value['addresse_BAN'][0])

upload_button = widgets.FileUpload(
    description='Upload csv file',
    accept='.csv',  # Accepted file extension e.g. '.txt', '.pdf', 'image/*', 'image/*,.pdf'
    multiple=False  # True to accept multiple files upload else False
)

find_button.on_click(on_find_button_clicked)        

get_csv_button = LoadedButton(
    value=1,
    description='Click me',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Click me',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
output_csv = widgets.Output(layout=widgets.Layout(border='1px solid black', overflow_y='auto', height='300px'))

def on_get_csv_button_clicked(b):
    with output_csv:
        fichier = upload_button.value['test2.csv']['content']        
        liv = find_livraison(fichier)
        b.value = qgrid.show_grid(liv, show_toolbar=True)
        display(b.value)

get_csv_button.on_click(on_get_csv_button_clicked)

draw_map = LoadedButton(
    value=1,
    description='Génerer Carte',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Génerer Carte',
    icon='check' # (FontAwesome names without the `fa-` prefix)
)
output_map = widgets.Output(layout=widgets.Layout(border='1px solid black', overflow_y='auto', height='600px'))
output_link = widgets.Output(layout=widgets.Layout(border='1px solid black', overflow_y='auto', height='30px'))

def on_get_draw_button_clicked(b):
    b.value = draw_map_from_liv(find_button, get_csv_button)
    with output_map:
        display(b.value)
    with output_link:
        map_html = "map_tournees.html"
        folium_map = draw_map.value
        folium_map.save(map_html)
        html_string = folium_map.get_root().render()    
        b64 = base64.b64encode(html_string.encode())
        title = "Download Map file"
        filename = "map_tournees.html"
        payload = b64.decode()    
        html = '<a download="{filename}" href="data:text/csv;base64,{payload}" target="_blank">{title}</a>'
        html = html.format(payload=payload,title=title,filename=filename)
        display(HTML(html))        

draw_map.on_click(on_get_draw_button_clicked)        

export_features = widgets.HBox([Nom_Magasin, Adresse_Magasin])
export_features2 = widgets.HBox([CP_Magasin, Ville_Magasin])
export_features3 = widgets.HBox([upload_button, get_csv_button])
export_features4 = widgets.HBox([draw_map, output_link])
display(export_features, export_features2, find_button, output_find, export_features3, output_csv, export_features4, output_map)




NameError: name 'save_map2' is not defined